# Creating a Pydantic Model

To define a Pydantic model, we have to inherit from the `BaseModel` class.

In [1]:
from pydantic import BaseModel

Think of a Pydantic model as a data structure containing named fields.

Furthermore, we add information about the expected type of each field, using type hints.

In [6]:
class Person(BaseModel):
    first_name: str
    last_name: str
    age: int
    nationality: str = "NL"

This is a basic Pydantic model. It is a Pydantic model because it **inherits** from `BaseModel`, but it is also a regular Python class, so you can choose to add your own methods properties, etc to it if you so wish.

It has three fields, and each field has been provided a type hint indicating its expected type.

Since Pydantic leverages Python's type hinting system, you can use any Python type hint you want here, including custom types, generics, etc.

Now that we have a model defined, we can create instances of the model. We have three basic ways of doing this.

In [8]:
p = Person(first_name="Evariste", last_name="Galois", age=20)

Pydantic automatically provides `str` and `repr` representations:

In [9]:
str(p)

"first_name='Evariste' last_name='Galois' age=20 nationality='NL'"

In [10]:
repr(p)

"Person(first_name='Evariste', last_name='Galois', age=20, nationality='NL')"

We can inspect the fields in a Pydantic model using the `model_fields` property:

In [11]:
p.model_fields

/tmp/ipython-input-1166466245.py:1: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  p.model_fields


{'first_name': FieldInfo(annotation=str, required=True),
 'last_name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=True),
 'nationality': FieldInfo(annotation=str, required=False, default='NL')}

In [17]:
p.__class__.model_fields["nationality"].default

'NL'

You'll notice that each field is listed, and one interesting thing is that `required=True` attached to each field.

And indeed, if we were to try and create a model instance without passing in a value for any of those fields, we would get an exception.

Pydantic raise the `ValidationError` exception whenever something goes wrong trying to create a model instance.

In [18]:
from pydantic import ValidationError

Now let's see what happens when we don't provide a required field:

In [19]:
try:
    Person(last_name='Galois')
except ValidationError as ex:
    print(ex)

2 validation errors for Person
first_name
  Field required [type=missing, input_value={'last_name': 'Galois'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
age
  Field required [type=missing, input_value={'last_name': 'Galois'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing


As you can see, Pydantic reports back on **all** the validation errors it encounters.

By default, Pydantic does not stop validating data when it encounters the first validation issue. Instead it continues validating everything, and then returns a complete list of all the validation errors.

This can be very useful for REST APIs, where you typically want to let an API caller know **everything** that was wrong with their JSON payload.

I mentioned that Pydantic classes are just regular Python classes. They obtain their special functionality by inheriting from `BaseModel`, but they are standard classes. Which means we can add properties and methods to it.

In [ ]:
class Person(BaseModel):
    first_name: str
    last_name: str
    age: int

    @property
    def display_name(self):
        return f"{self.first_name} {self.last_name[0]}"

In [ ]:
p = Person(first_name="Evariste", last_name="Galois", age=20)
p

Person(first_name='Evariste', last_name='Galois', age=20)

In [ ]:
p.display_name

'Evariste G'

Just like a normal object instance, you can also access the fields themselves using dot notation:

In [ ]:
p.last_name

'Galois'

And, by default, you can also change field values:

In [ ]:
p.age = 21

In [ ]:
p

Person(first_name='Evariste', last_name='Galois', age=21)

Now, we have to be a bit careful here. Pydantic will perform validation when it loads data (**deserializes** data) to create a model instance.

In particular, we have type hints attached to each field:

In [ ]:
try:
    Person(first_name='Evariste', last_name='Galois', age='twenty')
except ValidationError as ex:
    print(ex)

1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='twenty', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/int_parsing


As you can see, our incorrect input for `age` is trapped by Pydantic.

However, by default, this does not happen when we set a field value once the model instance has been created:

In [ ]:
p

Person(first_name='Evariste', last_name='Galois', age=21)

In [ ]:
p.age = "twenty"

In [ ]:
p

Person(first_name='Evariste', last_name='Galois', age='twenty')

See? The change went through.

In a later video, I'll show you how we can configure our Pydantic model so that doing this will also raise a `ValidationError` exception.